In [68]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [69]:
### data import need to change url
data = pd.read_csv("/Users/ruijiezhang/capstone/traffic_cleaned_forPython.csv", parse_dates=['day'])
data.set_index('day', inplace=True)
data.sort_index(inplace=True)

### One hot encoding
data = pd.get_dummies(data, dtype="bool")

### train test spilit
target_col = 'CORE_VL_NbEntry'
train_ratio = 0.7
val_ratio = 0.2
test_ratio = 0.1
train_size = int(train_ratio * len(data))
val_size = int(val_ratio * len(data))
test_size = len(data) - train_size - val_size
train_data = data.iloc[:train_size]
val_data = data.iloc[train_size:train_size+val_size]
test_data = data.iloc[-test_size:]
feature_cols = [col for col in data.columns if col != target_col]
X_train, y_train = train_data[feature_cols], train_data[target_col]
X_val, y_val = val_data[feature_cols], val_data[target_col]
X_test, y_test = test_data[feature_cols], test_data[target_col]

In [71]:
## Dataset assignment
train_lgb = lgb.Dataset(X_train, label=y_train, free_raw_data=False)
val_lgb = lgb.Dataset(X_val, label=y_val, reference=train_lgb, free_raw_data=False)



In [77]:
### hyperparameter 
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'n_estimators': 100
}

In [78]:
### model fitting
model = lgb.train(params, train_lgb, valid_sets=[val_lgb], early_stopping_rounds=10)

/Users/ruijiezhang/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:148: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 655
[LightGBM] [Info] Number of data points in the train set: 132265, number of used features: 176
[LightGBM] [Info] Start training from score 46.507330
[1]	valid_0's rmse: 37.6295
Training until validation scores don't improve for 10 rounds
[2]	valid_0's rmse: 36.6746
[3]	valid_0's rmse: 35.8427
[4]	valid_0's rmse: 35.1047
[5]	valid_0's rmse: 34.4794
[6]	valid_0's rmse: 33.9346
[7]	valid_0's rmse: 33.4803
[8]	valid_0's rmse: 33.0951
[9]	valid_0's rmse: 32.7928
[10]	valid_0's rmse: 32.5317
[11]	valid_0's rmse: 32.333
[12]	valid_0's rmse: 32.1882
[13]	valid_0's rmse: 32.0868
[14]	valid_0's rmse: 32.0248
[15]	valid_0's rmse: 31.9812
[16]	valid_0's rmse: 31

In [79]:
# Evaluate model on test data
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"Test RMSE: {rmse:.2f}")

Test RMSE: 24.69


In [80]:
y_pred.mean()

48.867580194976625